In [ ]:
# CURRENT --> ADD TESTs / Accuracy reports
# Remove 15 percent of each userid's rows
# train on the 85 percent
# test on the 15 percent that has NOT been used in training
# -problem is that custom functions are used instead of sklearn classifiers
# -so need to figure out the interface between the two and emulate it
# -OR put functions in custom classifier class, needs some methods defined like fit(), train(), ...

# Python User-User Collaborative Filtering Recommender System

# Datafield has 'user_ids' and their 'rating' (1-10) of various 'game_ids'
# Prediction of unrated games for each user_id is made using:
# k-nearest neighbors found using cosine-similarity
# predicted rating based on those near-neighbors' rating weighted by cosine-similarity users

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('inputs/boardgame-elite-users.csv')
rating_ptable = df.pivot_table(index='userID', columns='gameID', values='rating') 

rating_ptable = rating_ptable.fillna(0)

In [2]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(rating_ptable, test_size=.15)

In [3]:
print(train.head)
print('XXXXXXXXX')
print(test.head)

<bound method NDFrame.head of gameID  3       5       10      11      12      13      18      41      \
userID                                                                   
26112     3.00   10.00    0.00    5.00    9.00    8.00    7.00    3.00   
127141    8.00    6.00    3.00    5.00    8.00    6.00    6.00    0.00   
162252    7.00    8.00    7.00    6.00    5.00    6.00    7.00    6.00   
96422     0.00    7.30    0.00    6.70    7.50    7.80    0.00    0.00   
58562     7.00    6.00    8.00    7.00    8.00    8.00    6.00    7.00   
7100      7.00    9.00    7.00    7.00    7.00   10.00    5.00    8.00   
388       8.00    7.00    8.00    8.00    8.00    7.00    8.00    9.00   
272       1.00    1.00    1.00    0.00    1.00    7.50    1.00    4.00   
155112    7.50    7.50    0.00    9.00    7.50    7.00    7.00    8.00   
99214     3.00    2.00    7.00    6.00    9.00    3.00    2.00    7.00   
114589    6.00    5.00    7.00    8.00    9.00    9.00    9.00    7.00   
169530  

In [4]:
from sklearn.preprocessing import normalize

norm_train = normalize(train)

In [5]:
from sklearn.metrics.pairwise import cosine_similarity

sim = cosine_similarity(norm_train)

In [ ]:
nbor_indx = sim.argsort()[:,::-1]

# HELPER FUNCTIONS
def get_users_that_rated(gameid):
    return train[gameid].where(train[gameid]>0).dropna().index.tolist()
    
def get_k_nearest_neighbors(userid, gameid, k):
    assert(k>1)
    pot_nbors = get_users_that_rated(gameid)
    userid_indx = train.index.get_loc(userid)
    
    sim_userIDs = []
    for x in nbor_indx[userid_indx]:
        if train.index[x] in pot_nbors:
            sim_userIDs.append(train.index[x])
        if len(sim_userIDs)>k:
            break
    return sim_userIDs[1:]

In [ ]:
# Takes userID, gameID, optional k
# Returns predicted rating for gameID
def predict_rating(uid, gameid, k=10):
    sim_uids = get_k_nearest_neighbors(uid, gameid, k)
    accm_score = 0.0
    accm_weight = 0.0
    for nbor_id in sim_uids:
        sim_indx_x = train.index.get_loc(uid)
        sim_indx_y = train.index.get_loc(nbor_id)
        accm_score += (rating_ptable.loc[nbor_id][gameid] * sim[sim_indx_x][sim_indx_y])
        accm_weight += sim[sim_indx_x][sim_indx_y]
    score = accm_score / accm_weight
    return score

predict_rating(5480, 97903)